Technological Institute of the Philippines | Quezon City - Computer Engineering
--- | ---
Course Code: | CPE 313
Code Title: |  Advanced Machine Learning and Deep Learning
2st Semester | AY 2024-2025
**ACTIVITY NO. 7** | **Performing_Face_Recognition**
**Name** | Detchosa, Ralph Christian D.
**Section** | CPE32S3
**Date Performed**: | 02/21/2025
**Date Submitted**: | 02/21/2025
**Instructor**: | Engr. Roman M. Richard

<hr>

## 1. Objectives

This activity aims to enable students to perform data preparation and face recognition on their own generated dataset.

## 2. Intended Learning Outcomes (ILOs)
After this activity, the students should be able to:
* Utilize data preparation techniques for images.
* Perform Face Recognition using multiple algorithms.
* Evaluate the performance of different algorithms.

## 3. Procedures and Outputs

### Preparing the training data

Now that we have our data, we need to load these sample pictures into our face recognition algorithms. All face recognition algorithms take two parameters in their `train()` method: an array of images and an array of labels. What do these labels represent? They are the IDs of a certain individual/face so that when face recognition is performed, we not only know the person was recognized but also who—among the many people available in our database—the person is.

To do that, we need to create a comma-separated value (CSV) file, which will contain the path to a sample picture followed by the ID of that person.

**Include a Screenshot of Your Dataset Here**

---

### Loading the data and recognizing faces

Next up, we need to load these two resources (the array of images and CSV file) into the face recognition algorithm, so it can be trained to recognize our face. To do this, we build a function that reads the CSV file and—for each line of the file—loads the image at the corresponding path into the images array and the ID into the labels array.

In [10]:
import numpy as np
import os
import errno
import sys
import cv2

def read_images(path, sz=None):
  c = 0
  X, y = [], []

  for dirname, dirnames, filenames in os.walk(path):
    for subdirname in dirnames:
      subject_path = os.path.join(dirname, subdirname)
      for filename in os.listdir(subject_path):
        try:
          if(filename == ".directory"):
            continue
          filepath = os.path.join(subject_path, filename)
          im = cv2.imread(os.path.join(subject_path, filename), cv2.IMREAD_GRAYSCALE)

          # Resize the images to the prescribed size
          if (sz is not None):
            im = cv2.resize(im, (200,200))

          X.append(np.asarray(im, dtype=np.uint8))
          y.append(c)

        except IOError as e:
          print(f"I/O Error({e.errno}): {e.strerror}")
        except:
          print("Unexpected error:", sys.exc_info()[0])
          raise
      c = c+1
  return [X, y]

dataset = read_images(r"C:\Users\qwvan\Downloads\dataset")
print(dataset)

[[array([[ 47,  43,  47, ...,  57,  59,  64],
       [ 50,  50,  54, ...,  58,  62,  66],
       [ 52,  55,  58, ...,  59,  63,  67],
       ...,
       [136, 139, 139, ...,  71,  73,  75],
       [135, 136, 136, ...,  77,  78,  79],
       [135, 133, 133, ...,  74,  74,  74]], dtype=uint8), array([[ 44,  45,  49, ...,  49,  53,  57],
       [ 40,  45,  51, ...,  52,  55,  58],
       [ 39,  47,  55, ...,  52,  55,  57],
       ...,
       [134, 134, 133, ...,  56,  56,  56],
       [130, 131, 130, ...,  55,  53,  51],
       [127, 129, 128, ...,  54,  50,  47]], dtype=uint8), array([[50, 49, 46, ..., 66, 65, 65],
       [49, 48, 45, ..., 66, 65, 66],
       [48, 47, 45, ..., 65, 65, 66],
       ...,
       [51, 51, 53, ..., 37, 38, 39],
       [52, 52, 55, ..., 37, 38, 38],
       [53, 53, 55, ..., 38, 38, 38]], dtype=uint8), array([[43, 44, 44, ..., 63, 61, 64],
       [44, 44, 44, ..., 64, 62, 65],
       [44, 44, 43, ..., 65, 63, 65],
       ...,
       [42, 41, 39, ..., 39, 38, 37

**Question: Run the function above on your generated dataset. Provide an analysis and note all the challenges you have encountered running this code.**

- error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv_contrib\modules\face\src\eigen_faces.cpp:62: error: (-5:Bad argument) Empty training data was given. You'll need more than one sample to learn a model. in function 'cv::face::Eigenfaces::train'




---

### Performing Face Recognition Algorithms

Here is a sample script for testing the Face Recognition Algorithm. In this section, we're going to follow the same process but with different algorithms for face recognitions, namely:
- Eigenface Recognition
- Fisherface Recognition
- Local Binary Pattern Histograms (LBPH) Recognition

In [21]:
def face_rec():
  names = ['rave', 'ralph'] # Put your names here for faces to recognize

  if len(sys.argv) < 2:
    print("USAGE: facerec_demo.py </path/to/images> [</path/to/store/images/at>]")
    sys.exit()

  [X, y] = read_images(r"C:\Users\qwvan\Downloads\Detchosa_Activity_7_Performing_Face_Recognition\dataset", (200, 200))
  y = np.asarray(y, dtype=np.int32)

  model = cv2.face.EigenFaceRecognizer_create()
  model.train(X, y)

  camera = cv2.VideoCapture(0)
  face_cascade = cv2.CascadeClassifier(r"C:\Users\qwvan\Downloads\haarcascade_frontalface_default.xml")

  while True:
    ret, img = camera.read()
    if not ret:
      break

    faces = face_cascade.detectMultiScale(img, 1.3, 5)

    for (x, y, w, h) in faces:
      cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
      gray = cv2.cvtColor(img[y:y + h, x:x + w], cv2.COLOR_BGR2GRAY)
      roi = cv2.resize(gray, (200, 200), interpolation=cv2.INTER_LINEAR)

      try:
        params = model.predict(roi)
        label = names[params[0]]
        cv2.putText(img, label + ", " + str(params[1]), (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
      except:
        continue

    cv2.imshow("camera", img)
    if cv2.waitKey(1) & 0xFF == ord("q"):
      break

  camera.release()
  cv2.destroyAllWindows()

if __name__ == "__main__":
  face_rec()


**Question: Provide an analysis of the sample script for the process using the Eigenface Model. What is the sample code doing? Are you able to troubleshoot any problems encountered?**

---
Perform the remaining face recognition techniques by using the same (or modified) process from the sample code:

- `model = cv2.face.createFisherFaceRecognizer()`
- `model = cv2.face.createLBPHFaceRecognizer()`

- `model = cv2.face.createFisherFaceRecognizer()`

In [ ]:
import numpy as np
import os
import errno
import sys
import cv2

def read_images(path, sz=None):
  c = 0
  X, y = [], []

  for dirname, dirnames, filenames in os.walk(path):
    for subdirname in dirnames:
      subject_path = os.path.join(dirname, subdirname)
      for filename in os.listdir(subject_path):
        try:
          if(filename == ".directory"):
            continue
          filepath = os.path.join(subject_path, filename)
          im = cv2.imread(os.path.join(subject_path, filename), cv2.IMREAD_GRAYSCALE)

          # Resize the images to the prescribed size
          if (sz is not None):
            im = cv2.resize(im, (200,200))

          X.append(np.asarray(im, dtype=np.uint8))
          y.append(c)

        except IOError as e:
          print(f"I/O Error({e.errno}): {e.strerror}")
        except:
          print("Unexpected error:", sys.exc_info()[0])
          raise
      c = c+1
  return [X, y]

dataset = read_images(r"C:\Users\qwvan\Downloads\dataset")

def face_rec():
  names = ['rave', 'ralph'] # Put your names here for faces to recognize

  if len(sys.argv) < 2:
    print("USAGE: facerec_demo.py </path/to/images> [</path/to/store/images/at>]")
    sys.exit()

  [X, y] = read_images(r"C:\Users\qwvan\Downloads\Detchosa_Activity_7_Performing_Face_Recognition\dataset", (200, 200))
  y = np.asarray(y, dtype=np.int32)

  model = cv2.face.FisherFaceRecognizer_create()
  model.train(X, y)

  camera = cv2.VideoCapture(0)
  face_cascade = cv2.CascadeClassifier(r"C:\Users\qwvan\Downloads\haarcascade_frontalface_default.xml")

  while True:
    ret, img = camera.read()
    if not ret:
      break

    faces = face_cascade.detectMultiScale(img, 1.3, 5)

    for (x, y, w, h) in faces:
      cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
      gray = cv2.cvtColor(img[y:y + h, x:x + w], cv2.COLOR_BGR2GRAY)
      roi = cv2.resize(gray, (200, 200), interpolation=cv2.INTER_LINEAR)

      try:
        params = model.predict(roi)
        label = names[params[0]]
        cv2.putText(img, label + ", " + str(params[1]), (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
      except:
        continue

    cv2.imshow("camera", img)
    if cv2.waitKey(1) & 0xFF == ord("q"):
      break

  camera.release()
  cv2.destroyAllWindows()

if __name__ == "__main__":
  face_rec()


[[array([[ 47,  43,  47, ...,  57,  59,  64],
       [ 50,  50,  54, ...,  58,  62,  66],
       [ 52,  55,  58, ...,  59,  63,  67],
       ...,
       [136, 139, 139, ...,  71,  73,  75],
       [135, 136, 136, ...,  77,  78,  79],
       [135, 133, 133, ...,  74,  74,  74]], dtype=uint8), array([[ 44,  45,  49, ...,  49,  53,  57],
       [ 40,  45,  51, ...,  52,  55,  58],
       [ 39,  47,  55, ...,  52,  55,  57],
       ...,
       [134, 134, 133, ...,  56,  56,  56],
       [130, 131, 130, ...,  55,  53,  51],
       [127, 129, 128, ...,  54,  50,  47]], dtype=uint8), array([[50, 49, 46, ..., 66, 65, 65],
       [49, 48, 45, ..., 66, 65, 66],
       [48, 47, 45, ..., 65, 65, 66],
       ...,
       [51, 51, 53, ..., 37, 38, 39],
       [52, 52, 55, ..., 37, 38, 38],
       [53, 53, 55, ..., 38, 38, 38]], dtype=uint8), array([[43, 44, 44, ..., 63, 61, 64],
       [44, 44, 44, ..., 64, 62, 65],
       [44, 44, 43, ..., 65, 63, 65],
       ...,
       [42, 41, 39, ..., 39, 38, 37

- model = cv2.face.createLBPHFaceRecognizer()

In [ ]:
import numpy as np
import os
import errno
import sys
import cv2

def read_images(path, sz=None):
  c = 0
  X, y = [], []

  for dirname, dirnames, filenames in os.walk(path):
    for subdirname in dirnames:
      subject_path = os.path.join(dirname, subdirname)
      for filename in os.listdir(subject_path):
        try:
          if(filename == ".directory"):
            continue
          filepath = os.path.join(subject_path, filename)
          im = cv2.imread(os.path.join(subject_path, filename), cv2.IMREAD_GRAYSCALE)

          # Resize the images to the prescribed size
          if (sz is not None):
            im = cv2.resize(im, (200,200))

          X.append(np.asarray(im, dtype=np.uint8))
          y.append(c)

        except IOError as e:
          print(f"I/O Error({e.errno}): {e.strerror}")
        except:
          print("Unexpected error:", sys.exc_info()[0])
          raise
      c = c+1
  return [X, y]

dataset = read_images(r"C:\Users\qwvan\Downloads\dataset")

def face_rec():
  names = ['rave', 'ralph'] # Put your names here for faces to recognize

  if len(sys.argv) < 2:
    print("USAGE: facerec_demo.py </path/to/images> [</path/to/store/images/at>]")
    sys.exit()

  [X, y] = read_images(r"C:\Users\qwvan\Downloads\Detchosa_Activity_7_Performing_Face_Recognition\dataset", (200, 200))
  y = np.asarray(y, dtype=np.int32)

  model = cv2.face.LBPHFaceRecognizer_create()
  model.train(X, y)

  camera = cv2.VideoCapture(0)
  face_cascade = cv2.CascadeClassifier(r"C:\Users\qwvan\Downloads\haarcascade_frontalface_default.xml")

  while True:
    ret, img = camera.read()
    if not ret:
      break

    faces = face_cascade.detectMultiScale(img, 1.3, 5)

    for (x, y, w, h) in faces:
      cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
      gray = cv2.cvtColor(img[y:y + h, x:x + w], cv2.COLOR_BGR2GRAY)
      roi = cv2.resize(gray, (200, 200), interpolation=cv2.INTER_LINEAR)

      try:
        params = model.predict(roi)
        label = names[params[0]]
        cv2.putText(img, label + ", " + str(params[1]), (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
      except:
        continue

    cv2.imshow("camera", img)
    if cv2.waitKey(1) & 0xFF == ord("q"):
      break

  camera.release()
  cv2.destroyAllWindows()

if __name__ == "__main__":
  face_rec()


**Question: The `predict()` method returns a two-element array. Provide your analysis of the two returned values and their important ince this application.**

## 4. Supplementary Activity

Your accomplisment of the tasks below contribute to the achievement of ILO1, ILO2, and ILO3 for this module.

---

Tasks:
1. Create a new dataset for testing, this dataset must include the following:
  - The same person/s that the model has to recognize.
  - Different person/s that the model should not recognize.
2. For each model, perform 20 tests. Document the testing performed and provide observations.
3. Conclude on the performed tests by providing your evaluation of the performance of the models.

In [44]:
import os
import cv2
import numpy as np

def read_images(path, size=(200, 200)):
    X, y = [], []
    label = 0
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith('.jpg') or filename.endswith('.png'):
                filepath = os.path.join(dirname, filename)
                img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img = cv2.resize(img, size)
                    X.append(np.asarray(img, dtype=np.uint8))
                    y.append(label)
        label += 1
    return [X, y]

def test_model(model, test_path):
    [X_test, y_test] = read_images(test_path)
    if len(y_test) == 0:
        print("Error: Test dataset is empty.")
        return 0
    y_test = np.asarray(y_test, dtype=np.int32)
    correct = 0
    for i, img in enumerate(X_test):
        label, confidence = model.predict(img)
        if label == y_test[i]:
            correct += 1
    accuracy = correct / len(y_test)
    return accuracy

def evaluate_model(model, model_name, train_path, test_path):
    [X_train, y_train] = read_images(train_path)
    if len(y_train) == 0:
        print(f"Error: Training dataset is empty for {model_name}.")
        return
    y_train = np.asarray(y_train, dtype=np.int32)

    model.train(X_train, y_train)

    accuracies = []
    for _ in range(20):
        accuracy = test_model(model, test_path)
        if accuracy == 0:
            print(f"Skipping test due to empty test dataset for {model_name}.")
            continue
        accuracies.append(accuracy)
        print(f"{model_name} Test {_ + 1}: Accuracy = {accuracy * 100:.2f}%")

    if accuracies:
        avg_accuracy = np.mean(accuracies)
        print(f"{model_name} Average Accuracy: {avg_accuracy * 100:.2f}%")
    else:
        print(f"No valid tests were performed for {model_name}.")

def main():
    train_path = r"C:\Users\qwvan\Downloads\Detchosa_Activity_7_Performing_Face_Recognition\dataset"
    test_path = r"C:\Users\qwvan\Downloads\Detchosa_Activity_7_Performing_Face_Recognition\test_dataset"

    # Test LBPHFaceRecognizer
    lbph_model = cv2.face.LBPHFaceRecognizer_create()
    evaluate_model(lbph_model, "LBPHFaceRecognizer", train_path, test_path)

    # Test FisherFaceRecognizer
    fisher_model = cv2.face.FisherFaceRecognizer_create()
    evaluate_model(fisher_model, "FisherFaceRecognizer", train_path, test_path)

    # Test EigenFaceRecognizer
    eigen_model = cv2.face.EigenFaceRecognizer_create()
    evaluate_model(eigen_model, "EigenFaceRecognizer", train_path, test_path)

if __name__ == "__main__":
    main()

LBPHFaceRecognizer Test 1: Accuracy = 80.00%
LBPHFaceRecognizer Test 2: Accuracy = 80.00%
LBPHFaceRecognizer Test 3: Accuracy = 80.00%
LBPHFaceRecognizer Test 4: Accuracy = 80.00%
LBPHFaceRecognizer Test 5: Accuracy = 80.00%
LBPHFaceRecognizer Test 6: Accuracy = 80.00%
LBPHFaceRecognizer Test 7: Accuracy = 80.00%
LBPHFaceRecognizer Test 8: Accuracy = 80.00%
LBPHFaceRecognizer Test 9: Accuracy = 80.00%
LBPHFaceRecognizer Test 10: Accuracy = 80.00%
LBPHFaceRecognizer Test 11: Accuracy = 80.00%
LBPHFaceRecognizer Test 12: Accuracy = 80.00%
LBPHFaceRecognizer Test 13: Accuracy = 80.00%
LBPHFaceRecognizer Test 14: Accuracy = 80.00%
LBPHFaceRecognizer Test 15: Accuracy = 80.00%
LBPHFaceRecognizer Test 16: Accuracy = 80.00%
LBPHFaceRecognizer Test 17: Accuracy = 80.00%
LBPHFaceRecognizer Test 18: Accuracy = 80.00%
LBPHFaceRecognizer Test 19: Accuracy = 80.00%
LBPHFaceRecognizer Test 20: Accuracy = 80.00%
LBPHFaceRecognizer Average Accuracy: 80.00%
FisherFaceRecognizer Test 1: Accuracy = 80.00

In [45]:
import os
import cv2
import numpy as np

def read_images(path, size=(200, 200)):
    X, y = [], []
    label = 0
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith('.jpg') or filename.endswith('.png'):
                filepath = os.path.join(dirname, filename)
                img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img = cv2.resize(img, size)
                    X.append(np.asarray(img, dtype=np.uint8))
                    y.append(label)
        label += 1
    return [X, y]

def test_model(model, test_path):
    [X_test, y_test] = read_images(test_path)
    if len(y_test) == 0:
        print("Error: Test dataset is empty.")
        return 0
    y_test = np.asarray(y_test, dtype=np.int32)
    correct = 0
    for i, img in enumerate(X_test):
        label, confidence = model.predict(img)
        print(f"Predicted: {label}, Actual: {y_test[i]}, Confidence: {confidence}")
        if label == y_test[i]:
            correct += 1
    accuracy = correct / len(y_test)
    return accuracy

def evaluate_model(model, model_name, train_path, test_path):
    [X_train, y_train] = read_images(train_path)
    if len(y_train) == 0:
        print(f"Error: Training dataset is empty for {model_name}.")
        return
    y_train = np.asarray(y_train, dtype=np.int32)

    model.train(X_train, y_train)

    accuracies = []
    for _ in range(20):
        accuracy = test_model(model, test_path)
        if accuracy == 0:
            print(f"Skipping test due to empty test dataset for {model_name}.")
            continue
        accuracies.append(accuracy)
        print(f"{model_name} Test {_ + 1}: Accuracy = {accuracy * 100:.2f}%")

    if accuracies:
        avg_accuracy = np.mean(accuracies)
        print(f"{model_name} Average Accuracy: {avg_accuracy * 100:.2f}%")
    else:
        print(f"No valid tests were performed for {model_name}.")

def main():
    train_path = r"C:\Users\qwvan\Downloads\Detchosa_Activity_7_Performing_Face_Recognition\dataset"
    test_path = r"C:\Users\qwvan\Downloads\Detchosa_Activity_7_Performing_Face_Recognition\test_dataset"

    # Test LBPHFaceRecognizer
    lbph_model = cv2.face.LBPHFaceRecognizer_create()
    evaluate_model(lbph_model, "LBPHFaceRecognizer", train_path, test_path)

    # Test FisherFaceRecognizer
    fisher_model = cv2.face.FisherFaceRecognizer_create()
    evaluate_model(fisher_model, "FisherFaceRecognizer", train_path, test_path)

    # Test EigenFaceRecognizer
    eigen_model = cv2.face.EigenFaceRecognizer_create()
    evaluate_model(eigen_model, "EigenFaceRecognizer", train_path, test_path)

if __name__ == "__main__":
    main()

Predicted: 1, Actual: 1, Confidence: 0.0
Predicted: 1, Actual: 1, Confidence: 0.0
Predicted: 2, Actual: 2, Confidence: 0.0
Predicted: 2, Actual: 2, Confidence: 0.0
Predicted: 2, Actual: 3, Confidence: 101.00606898114056
LBPHFaceRecognizer Test 1: Accuracy = 80.00%
Predicted: 1, Actual: 1, Confidence: 0.0
Predicted: 1, Actual: 1, Confidence: 0.0
Predicted: 2, Actual: 2, Confidence: 0.0
Predicted: 2, Actual: 2, Confidence: 0.0
Predicted: 2, Actual: 3, Confidence: 101.00606898114056
LBPHFaceRecognizer Test 2: Accuracy = 80.00%
Predicted: 1, Actual: 1, Confidence: 0.0
Predicted: 1, Actual: 1, Confidence: 0.0
Predicted: 2, Actual: 2, Confidence: 0.0
Predicted: 2, Actual: 2, Confidence: 0.0
Predicted: 2, Actual: 3, Confidence: 101.00606898114056
LBPHFaceRecognizer Test 3: Accuracy = 80.00%
Predicted: 1, Actual: 1, Confidence: 0.0
Predicted: 1, Actual: 1, Confidence: 0.0
Predicted: 2, Actual: 2, Confidence: 0.0
Predicted: 2, Actual: 2, Confidence: 0.0
Predicted: 2, Actual: 3, Confidence: 101.

## 5. Summary, Conclusions and Lessons Learned

In this activity, it introduces the face recognition techniques using the OpenCV library. Using my own dataset, and apply three different face recognition algorithms: Eigenface, Fisherface, and LPBH. Then evaluate the performance the model. I have learned the significance of data preparation and the impact of image quality on the performance of face recognition models.

<hr/>

***Proprietary Clause***

*Property of the Technological Institute of the Philippines (T.I.P.). No part of the materials made and uploaded in this learning management system by T.I.P. may be copied, photographed, printed, reproduced, shared, transmitted, translated, or reduced to any electronic medium or machine-readable form, in whole or in part, without the prior consent of T.I.P.*